In [1]:
import json
import os
import _jsonnet
import os

In [2]:
from seq2struct.commands.infer import Inferer
from seq2struct.datasets.spider import SpiderItem
from seq2struct.utils import registry

In [3]:
import torch

In [4]:
exp_config = json.loads(
    _jsonnet.evaluate_file(
        "experiments/spider-configs/spider-mBART50MtoM-large-en-pt-es-fr-train_en-pt-es-fr-eval.jsonnet"))


In [5]:
model_config_path = exp_config["model_config"]
model_config_args = exp_config.get("model_config_args")

In [6]:
infer_config = json.loads(
    _jsonnet.evaluate_file(
        model_config_path, 
        tla_codes={'args': json.dumps(model_config_args)}))

In [7]:
infer_config["model"]["encoder_preproc"]["db_path"] = "data/sqlite_files/"

In [8]:
inferer = Inferer(infer_config)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}


SpiderEncoderBartPreproc Model: facebook/mbart-large-50-many-to-many-mmt
SpiderEncoderBartPreproc Pretrained Checkpoint: models/mBART50MtoM-large/pretrained_checkpoint/pytorch_model.bin
BART load Model: facebook/mbart-large-50-many-to-many-mmt


In [9]:
model_dir = exp_config["logdir"] + "/bs=12,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1"
#checkpoint_step = exp_config["eval_steps"][0]
checkpoint_step = 42100

In [10]:
model_dir

'/mnt/Databases/nl2sql/gap-text2sql/mrat-sql-gap/logdir/mBART50MtoM-large-en-pt-es-fr-train/bs=12,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1'

In [11]:
model = inferer.load_model(model_dir, checkpoint_step)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'data/spider-en-pt-es-fr/mBART50MtoM-large-nl2code-1115,output_from=true,fs=2,emb=bart,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'bart_version': 'facebook/mbart-large-50-many-to-many-mmt', 'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'data/sqlite_files/', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'pretrained_checkpoint': 'models/mBART50MtoM-large/pretrained_checkpoint/pytorch_model.bin', 'save_path': 'data/spider-en-pt-es-fr/mBART50MtoM-large-nl2code-1115,output_from=true,fs=2,emb=bart,cvlink'}}


SpiderEncoderBart Model: facebook/mbart-large-50-many-to-many-mmt
SpiderEncoderBart Pretrained Checkpoint: models/mBART50MtoM-large/pretrained_checkpoint/pytorch_model.bin


Some weights of the model checkpoint at facebook/mbart-large-50-many-to-many-mmt were not used when initializing MBartModel: ['final_logits_bias', 'lm_head.weight']
- This IS expected if you are initializing MBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Parameter containing:
tensor([[-0.0428, -0.0166,  0.0040,  ..., -0.0377,  0.0135,  0.0064],
        [-0.0486,  0.0342,  0.0125,  ..., -0.0298,  0.0023, -0.0117],
        [-0.0438, -0.0331, -0.0250,  ..., -0.0430,  0.0212, -0.0113],
        ...,
        [-0.0331, -0.0959, -0.0164,  ..., -0.0314,  0.0189, -0.0045],
        [-0.0296, -0.0487,  0.0050,  ...,  0.0017, -0.0259,  0.0182],
        [-0.0449, -0.0099, -0.0020,  ..., -0.0484,  0.0027, -0.0029]],
       requires_grad=True)
Updated the model with models/mBART50MtoM-large/pretrained_checkpoint/pytorch_model.bin
Parameter containing:
tensor([[-0.0428, -0.0166,  0.0040,  ..., -0.0377,  0.0135,  0.0064],
        [-0.0486,  0.0342,  0.0125,  ..., -0.0298,  0.0023, -0.0117],
        [-0.0438, -0.0331, -0.0250,  ..., -0.0430,  0.0212, -0.0113],
        ...,
        [ 0.0103,  0.0193, -0.0229,  ...,  0.0224,  0.0545, -0.0211],
        [-0.0350, -0.0251,  0.0002,  ..., -0.0259,  0.0119,  0.0058],
        [ 0.0126, -0.0247, -0.0331,  ...,  0

In [12]:
from seq2struct.datasets.spider_lib.preprocess.get_tables import dump_db_json_schema
from seq2struct.datasets.spider import load_tables_from_schema_dict

In [13]:
db_id = "singer"

In [14]:
my_schema = dump_db_json_schema("data/sqlite_files/{db_id}/{db_id}.sqlite".format(db_id=db_id), db_id)

In [15]:
from seq2struct.utils.api_utils import refine_schema_names

In [16]:
my_schema

{'db_id': 'singer',
 'table_names_original': ['singer', 'song'],
 'table_names': ['singer', 'song'],
 'column_names_original': [(-1, '*'),
  (0, 'Singer_ID'),
  (0, 'Name'),
  (0, 'Birth_Year'),
  (0, 'Net_Worth_Millions'),
  (0, 'Citizenship'),
  (1, 'Song_ID'),
  (1, 'Title'),
  (1, 'Singer_ID'),
  (1, 'Sales'),
  (1, 'Highest_Position')],
 'column_names': [(-1, '*'),
  (0, 'singer id'),
  (0, 'name'),
  (0, 'birth year'),
  (0, 'net worth millions'),
  (0, 'citizenship'),
  (1, 'song id'),
  (1, 'title'),
  (1, 'singer id'),
  (1, 'sales'),
  (1, 'highest position')],
 'column_types': ['text',
  'number',
  'text',
  'number',
  'number',
  'text',
  'number',
  'text',
  'number',
  'number',
  'number'],
 'primary_keys': [1, 6],
 'foreign_keys': [[8, 1]]}

In [17]:
# If you want to change your schema name, then run this; Otherwise you can skip this.
#refine_schema_names(my_schema)

In [18]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(my_schema)

In [19]:
schema.keys()

dict_keys(['singer'])

In [20]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": "data/sqlite_files/"
})

In [21]:
for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

In [22]:
spider_schema = dataset.schemas[db_id]

In [23]:
def infer(question):
    data_item = SpiderItem(
            text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
            code=None,
            schema=spider_schema,
            orig_schema=spider_schema.orig,
            orig={"question": question}
        )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        output = inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
    return output[0]["inferred_code"]

In [24]:
code = infer("How many singers are there?")
print(code)

Código do T5, mas na execução BART
Question: ['<s>', '▁How', '▁many', '▁sing', 'ers', '▁are', '▁there', '▁?', '</s>']
Columns: [['▁*', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁name', '</s>'], ['▁birth', '▁year', '</s>'], ['▁net', '▁worth', '▁millions', '</s>'], ['▁citizen', 'ship', '</s>'], ['▁song', '▁id', '</s>'], ['▁title', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁sales', '</s>'], ['▁highest', '▁position', '</s>']]
Tables: [['▁sing', 'er', '</s>'], ['▁song', '</s>']]

column_indexes: [9, 11, 15, 17, 20, 24, 27, 30, 32, 36, 38]
column_indexes_2: [9, 13, 15, 18, 22, 25, 28, 30, 34, 36, 39]
table_indexes: [41, 44]
table_indexes_2: [42, 44]

SELECT Count(*) FROM singer


/home/marchanjo/miniconda3/envs/gap-text2sql/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [25]:
code = infer("Quantos cantores nós temos?")
print(code)

Código do T5, mas na execução BART
Question: ['<s>', '▁Quanto', 's', '▁can', 'tores', '▁nós', '▁temos', '▁?', '</s>']
Columns: [['▁*', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁name', '</s>'], ['▁birth', '▁year', '</s>'], ['▁net', '▁worth', '▁millions', '</s>'], ['▁citizen', 'ship', '</s>'], ['▁song', '▁id', '</s>'], ['▁title', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁sales', '</s>'], ['▁highest', '▁position', '</s>']]
Tables: [['▁sing', 'er', '</s>'], ['▁song', '</s>']]

column_indexes: [9, 11, 15, 17, 20, 24, 27, 30, 32, 36, 38]
column_indexes_2: [9, 13, 15, 18, 22, 25, 28, 30, 34, 36, 39]
table_indexes: [41, 44]
table_indexes_2: [42, 44]

SELECT Count(*) FROM singer


In [26]:
code = infer("¿Cuántos cantantes tenemos?")
print(code)

Código do T5, mas na execução BART
Question: ['<s>', '▁¿', 'Cu', 'án', 'tos', '▁cantante', 's', '▁tenemos', '▁?', '</s>']
Columns: [['▁*', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁name', '</s>'], ['▁birth', '▁year', '</s>'], ['▁net', '▁worth', '▁millions', '</s>'], ['▁citizen', 'ship', '</s>'], ['▁song', '▁id', '</s>'], ['▁title', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁sales', '</s>'], ['▁highest', '▁position', '</s>']]
Tables: [['▁sing', 'er', '</s>'], ['▁song', '</s>']]

column_indexes: [10, 12, 16, 18, 21, 25, 28, 31, 33, 37, 39]
column_indexes_2: [10, 14, 16, 19, 23, 26, 29, 31, 35, 37, 40]
table_indexes: [42, 45]
table_indexes_2: [43, 45]

SELECT Count(*) FROM singer


In [27]:
code = infer("Combien de chanteurs avons-nous?")
print(code)

Código do T5, mas na execução BART
Question: ['<s>', '▁Combi', 'en', '▁de', '▁chan', 'teurs', '▁avons', '-', 'nous', '▁?', '</s>']
Columns: [['▁*', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁name', '</s>'], ['▁birth', '▁year', '</s>'], ['▁net', '▁worth', '▁millions', '</s>'], ['▁citizen', 'ship', '</s>'], ['▁song', '▁id', '</s>'], ['▁title', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁sales', '</s>'], ['▁highest', '▁position', '</s>']]
Tables: [['▁sing', 'er', '</s>'], ['▁song', '</s>']]

column_indexes: [11, 13, 17, 19, 22, 26, 29, 32, 34, 38, 40]
column_indexes_2: [11, 15, 17, 20, 24, 27, 30, 32, 36, 38, 41]
table_indexes: [43, 46]
table_indexes_2: [44, 46]

SELECT Count(*) FROM singer


In [28]:
code = infer("Quantos cantores estão disponiveis?")
print(code)

Código do T5, mas na execução BART
Question: ['<s>', '▁Quanto', 's', '▁can', 'tores', '▁estão', '▁dispon', 'i', 'veis', '▁?', '</s>']
Columns: [['▁*', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁name', '</s>'], ['▁birth', '▁year', '</s>'], ['▁net', '▁worth', '▁millions', '</s>'], ['▁citizen', 'ship', '</s>'], ['▁song', '▁id', '</s>'], ['▁title', '</s>'], ['▁sing', 'er', '▁id', '</s>'], ['▁sales', '</s>'], ['▁highest', '▁position', '</s>']]
Tables: [['▁sing', 'er', '</s>'], ['▁song', '</s>']]

column_indexes: [11, 13, 17, 19, 22, 26, 29, 32, 34, 38, 40]
column_indexes_2: [11, 15, 17, 20, 24, 27, 30, 32, 36, 38, 41]
table_indexes: [43, 46]
table_indexes_2: [44, 46]

SELECT Count(*) FROM singer
